In [21]:
import os
import re
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime, timedelta

# Load the data
ultra_marathon_loc = os.path.join(os.getcwd(), 'data', 'ultra_marathon.csv')
ultra_marathon = pd.read_csv(ultra_marathon_loc)
ultra_marathon.head(3)


/var/folders/3_/ymn7dyh91yn_mzg9vpnv7qq00000gn/T/ipykernel_21562/3725297650.py:10: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  ultra_marathon = pd.read_csv(ultra_marathon_loc)


,Year of event,Event dates,Event name,Event distance/length,Event number of finishers,Athlete performance,Athlete club,Athlete country,Athlete year of birth,Athlete gender,Athlete age category,Athlete average speed,Athlete ID
0,2018,06.01.2018,Selva Costera (CHI),50km,22,4:51:39 h,Tnfrc,CHI,1978.0,M,M35,10.286,0
1,2018,06.01.2018,Selva Costera (CHI),50km,22,5:15:45 h,Roberto Echeverría,CHI,1981.0,M,M35,9.501,1
2,2018,06.01.2018,Selva Costera (CHI),50km,22,5:16:44 h,Puro Trail Osorno,CHI,1987.0,M,M23,9.472,2


In [22]:
# Clean the data

# Rename columns
ultra_marathon.rename(columns={
    'Year of event':                'Year',         # int
    'Event dates':                  'Dates',        # [datetime]
    'Event name':                   'Event_name',   # str
    'Event distance/length':        'Length',       # str; contains km or hours
    'Event number of finishers':    'Finishers',    # int
    'Athlete performance':          'Performance',  # str; contains km or hours
    'Athlete club':                 'Club',         # str
    'Athlete country':              'Country',      # str
    'Athlete year of birth':        'Birth',        # float (contains significant nan values)
    'Athlete gender':               'Gender',       # str
    'Athlete age category':         'Age',          # int (contains significant nan values)
    'Athlete average speed':        'Average_speed',# float
    'Athlete ID':                   'Id'            # int
}, inplace=True)


def parseDates(given_string):
    # The possible formats are: '00.01.2000', '01.-05.01.2000', '31.01.-01.02.2000', '31.12.2000-01.01.2001'
    try:
        return [datetime.strptime(given_string, "%d.%m.%Y")]
    except:
        pass
    try:
        interval = given_string.split('-')
        for separation_length in [3, 6, 10]:
            if len(interval[0]) == separation_length:
                first_day = datetime.strptime(interval[0] + interval[1][separation_length:], "%d.%m.%Y")
                last_day = datetime.strptime(interval[1], "%d.%m.%Y")
                return [first_day + timedelta(days=i) for i in range((last_day - first_day).days + 1)]
    except:
        # None parsable & existent dates such as 00.01.2000
        return None
    

def convert_to_kilometers(given_string):
    if given_string == None or ("mi" not in given_string):
        return given_string
    else:
        return "{:.1f}km".format(float(given_string.replace('mi', '')) * 1.60934)


# Count how many X values are in gender
print((ultra_marathon['Gender']=='X').sum())

# Eliminate the X and nan values in gender
ultra_marathon = ultra_marathon[ultra_marathon['Gender'].notna() & (ultra_marathon['Gender'] != 'X')]

# Encode gender to 0 and 1
ultra_marathon['Gender'] = ultra_marathon['Gender'].map({'M': 0, 'F': 1})


# TODO: Uncomment lines

# Parse into appropiate data types
# pd.to_numeric(ultra_marathon['Year'], errors='coerce')
# ultra_marathon['Dates'] = ultra_marathon['Dates'].apply(lambda each: parseDates(each))
# ultra_marathon['Length'] = ultra_marathon['Length'].apply(convert_to_kilometers)
# pd.to_numeric(ultra_marathon['Finishers'], errors='coerce')
# pd.to_numeric(ultra_marathon['Birth'], errors='coerce')
# ultra_marathon['Age'] = ultra_marathon['Age'].str.slice(1,3)
# pd.to_numeric(ultra_marathon['Age'], errors='coerce')
# ultra_marathon['Age'] = ultra_marathon['Age'].str.extract(r'(\d+)')
# ultra_marathon['Age'] = pd.to_numeric(ultra_marathon['Age'], errors='coerce', downcast='integer')
# ultra_marathon['Average_speed'] = pd.to_numeric(ultra_marathon['Average_speed'], errors='coerce')
# ultra_marathon['Id'] = pd.to_numeric(ultra_marathon['Id'], errors='coerce', downcast='integer')



# print(ultra_marathon['Id'].unique()[:150])
# print(ultra_marathon['Id'].isna().sum())




# Drop all entries with null values, except for the 'Club' column (and maybe birth, age)

# ultra_marathon.dropna(inplace=True)
# print(ultra_marathon.isnull().sum())
# print(ultra_marathon[ultra_marathon['Finishers'] =="0"].sum())
# null_values = ultra_marathon[ultra_marathon['Event_name'].isnull() | (ultra_marathon['Event_name'] == '') | (ultra_marathon['Event_name'] == 'NULL') | (ultra_marathon['Event_name'] == 'NA')]







46


In [23]:
# Data exploration

print(ultra_marathon.info())

# Summary statistics
print(ultra_marathon.describe(include='all')) # include='all' includes non-numeric columns

# print unique values for categorical columns
print('Event name:', ultra_marathon['Event_name'].unique())
print('Length:', ultra_marathon['Length'].unique())
print('Club:', ultra_marathon['Club'].unique())
print('Country:', ultra_marathon['Country'].unique())
print('Gender:', ultra_marathon['Gender'].unique())

<class 'pandas.core.frame.DataFrame'>
Index: 7461142 entries, 0 to 7461194
Data columns (total 13 columns):
 #   Column         Dtype  
---  ------         -----  
 0   Year           int64  
 1   Dates          object 
 2   Event_name     object 
 3   Length         object 
 4   Finishers      int64  
 5   Performance    object 
 6   Club           object 
 7   Country        object 
 8   Birth          float64
 9   Gender         int64  
 10  Age            object 
 11  Average_speed  object 
 12  Id             int64  
dtypes: float64(1), int64(4), object(8)
memory usage: 796.9+ MB
None
                Year       Dates                 Event_name   Length  \
count   7.461142e+06     7461142                    7461142  7460089   
unique           NaN       14425                      26907     2159   
top              NaN  20.04.2019  Two Oceans Marathon (RSA)     50km   
freq             NaN       29176                     263574  1522593   
mean    2.011983e+03         NaN           